In [9]:
# Importer les bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator


In [2]:
# Initialiser la session Spark
spark = SparkSession.builder.appName("MushroomClassification").getOrCreate()


In [3]:
# Charger les données
data = spark.read.csv("mushrooms.csv", header=True, inferSchema=True)

# Nettoyer les noms des colonnes pour enlever les espaces
data = data.toDF(*[col.strip() for col in data.columns])

# Vérifier les colonnes
print("Colonnes disponibles dans le DataFrame :")
print(data.columns)

# Vérification si la colonne "class" est présente
if 'class' not in data.columns:
    raise ValueError("La colonne 'class' est manquante dans le DataFrame.")


Colonnes disponibles dans le DataFrame :
['class', 'cap-shape', 'cap-surface', 'cap-color', 'bruises', 'odor', 'gill-attachment', 'gill-spacing', 'gill-size', 'gill-color', 'stalk-shape', 'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring', 'stalk-color-below-ring', 'veil-type', 'veil-color', 'ring-number', 'ring-type', 'spore-print-color', 'population', 'habitat']


In [4]:
# Pré-traitement
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data) 
            for column in data.columns if column != 'class']
assembler = VectorAssembler(inputCols=[column+"_index" for column in data.columns if column != 'class'], 
                            outputCol="features")
label_indexer = StringIndexer(inputCol="class", outputCol="label").fit(data)

# Pipeline de transformation
pipeline = Pipeline(stages=indexers + [assembler, label_indexer])
data_transformed = pipeline.fit(data).transform(data)


In [5]:
# Diviser les données en ensembles d'entraînement et de test
train_data, test_data = data_transformed.randomSplit([0.8, 0.2], seed=1234)


In [6]:
# Initialiser une liste pour stocker les résultats
results = {}

# Modèles à tester
models = {
    "Logistic Regression": LogisticRegression(featuresCol="features", labelCol="label"),
    "Random Forest": RandomForestClassifier(featuresCol="features", labelCol="label"),
    "Gradient-Boosted Trees": GBTClassifier(featuresCol="features", labelCol="label"),
    "Decision Tree": DecisionTreeClassifier(featuresCol="features", labelCol="label"),
    "Naive Bayes": NaiveBayes(featuresCol="features", labelCol="label")
}


In [10]:
# Initialiser un dictionnaire pour stocker les métriques
metrics_results = {}

# Évaluer chaque modèle
for model_name, model in models.items():
    # Entraîner le modèle
    model_trained = model.fit(train_data)
    
    # Faire des prédictions
    predictions = model_trained.transform(test_data)

    # Évaluer avec ROC
    binary_evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="prediction", metricName="areaUnderROC")
    roc_auc = binary_evaluator.evaluate(predictions)

    # Évaluer avec précision, rappel et F1-score
    multi_evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
    multi_evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
    multi_evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
    
    precision = multi_evaluator_precision.evaluate(predictions)
    recall = multi_evaluator_recall.evaluate(predictions)
    f1_score = multi_evaluator_f1.evaluate(predictions)

    # Stocker les résultats dans le dictionnaire
    metrics_results[model_name] = {
        "ROC AUC": roc_auc,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1_score
    }


In [12]:
!pip install matplotlib

In [17]:
# Afficher les résultats
# for model_name, metrics in metrics_results.items():
#     print(f"{model_name} Metrics:")
#     print(f"  ROC AUC: {metrics['ROC AUC']:.4f}")
#     print(f"  Precision: {metrics['Precision']:.4f}")
#     print(f"  Recall: {metrics['Recall']:.4f}")
#     print(f"  F1 Score: {metrics['F1 Score']:.4f}")
import pandas as pd

# Créer un DataFrame à partir des résultats des métriques
metrics_df = pd.DataFrame(metrics_results).T  # Transpose pour avoir les modèles en lignes

# Afficher le tableau des métriques
print(metrics_df)
# Arrêter la session Spark
spark.stop()


                         ROC AUC  Precision    Recall  F1 Score
Logistic Regression     0.986211   0.986276  0.986250  0.986249
Random Forest           0.999369   0.999376  0.999375  0.999375
Gradient-Boosted Trees  0.999369   0.999376  0.999375  0.999375
Decision Tree           0.994318   0.994437  0.994375  0.994375
Naive Bayes             0.925980   0.927391  0.926250  0.926180


In [21]:
# Calculer la moyenne des métriques pour chaque colonne
mean_metrics = metrics_df.mean()

# Ajouter la ligne de moyenne au DataFrame
sorted_metrics_df.loc['Mean'] = mean_metrics

# Afficher le tableau avec la ligne de moyenne
print(sorted_metrics_df)


                         ROC AUC  Precision    Recall  F1 Score
Random Forest           0.999369   0.999376  0.999375  0.999375
Gradient-Boosted Trees  0.999369   0.999376  0.999375  0.999375
Decision Tree           0.994318   0.994437  0.994375  0.994375
Logistic Regression     0.986211   0.986276  0.986250  0.986249
Naive Bayes             0.925980   0.927391  0.926250  0.926180
Mean                    0.981049   0.981371  0.981125  0.981111
